### Load required libraries

In [6]:
import numpy as np
from scipy.integrate import odeint
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns


### Chikungunya model

In [7]:
def chikungunya_model(y, t, params):
    
    # Unpack parameters
    (
        b, beta_mh, beta_hm,
        contact_rate_1, contact_rate_2,
        vaccination_rate_1, vaccination_rate_2,
        severe_prob_1, severe_prob_2,
        chronic_prob_m, chronic_prob_s,
        gamma_m, gamma_s, gamma_c,
        sigma, sigma_m,
        mu, mu_m, Lambda_m,
        prop_age1_Im, prop_age1_Is, prop_age1_C, prop_age1_R, prop_age1_V
    ) = params
    
    # State variables
    S1, E1, S2, E2, Im, Is, C, R, V, Sm, Em, Im_mosq = y
    
    # Age-specific population sizes
    N1 = (S1 + E1 + prop_age1_Im*Im + prop_age1_Is*Is +
          prop_age1_C*C + prop_age1_R*R + prop_age1_V*V)
    
    N2 = (S2 + E2 + (1-prop_age1_Im)*Im + (1-prop_age1_Is)*Is +
          (1-prop_age1_C)*C + (1-prop_age1_R)*R + (1-prop_age1_V)*V)
    
    N_total = N1 + N2
    I_total = Im + Is

    # Forces of infection
    lambda_h = b * beta_mh * Im_mosq / N_total
    lambda_m = b * beta_hm * I_total / N_total
    
    lambda_h1 = lambda_h * contact_rate_1
    lambda_h2 = lambda_h * contact_rate_2
    
    # ODEs
    dS1 = mu*N1 - mu*S1 - lambda_h1*S1 - vaccination_rate_1*S1
    dE1 = lambda_h1*S1 - (sigma + mu)*E1
    
    dS2 = mu*N2 - mu*S2 - lambda_h2*S2 - vaccination_rate_2*S2
    dE2 = lambda_h2*S2 - (sigma + mu)*E2
    
    dIm = (
        sigma*(1-severe_prob_1)*E1 +
        sigma*(1-severe_prob_2)*E2 -
        (gamma_m + mu + chronic_prob_m*gamma_m)*Im
    )
    
    dIs = (
        sigma*severe_prob_1*E1 +
        sigma*severe_prob_2*E2 -
        (gamma_s + mu + chronic_prob_s*gamma_s)*Is
    )
    
    dC = (
        chronic_prob_m*gamma_m*Im +
        chronic_prob_s*gamma_s*Is -
        (gamma_c + mu)*C
    )
    
    dR = (
        (1-chronic_prob_m)*gamma_m*Im +
        (1-chronic_prob_s)*gamma_s*Is +
        gamma_c*C - mu*R
    )
    
    dV = vaccination_rate_1*S1 + vaccination_rate_2*S2 - mu*V
    
    # Mosquito equations
    dSm = Lambda_m - mu_m*Sm - lambda_m*Sm
    dEm = lambda_m*Sm - (sigma_m + mu_m)*Em
    dIm_mosq = sigma_m*Em - mu_m*Im_mosq
    
    return [
        dS1, dE1, dS2, dE2, dIm, dIs, dC, dR, dV,
        dSm, dEm, dIm_mosq
    ]


### Simulation Runner

In [8]:
def run_chikungunya_simulation(params, initial_conditions, time_points):
    
    # Solve ODE system
    sol = odeint(
        func=lambda y, t: chikungunya_model(y, t, params),
        y0=initial_conditions,
        t=time_points
    )
    
    # Convert to dataframe
    columns = [
        "S1", "E1", "S2", "E2", "Im", "Is", "C", "R", "V",
        "Sm", "Em", "Im_mosq"
    ]
    
    df = pd.DataFrame(sol, columns=columns)
    df["time"] = time_points
    
    return df


## PLOTTING FUNCTIONS
### Age-specific susceptible + Exposed

In [9]:
def plot_age_specific(df):
    plt.figure(figsize=(10,6))
    sns.lineplot(df, x="time", y="S1", label="S1 (<60)")
    sns.lineplot(df, x="time", y="E1", label="E1 (<60)")
    sns.lineplot(df, x="time", y="S2", label="S2 (60+)")
    sns.lineplot(df, x="time", y="E2", label="E2 (60+)")
    
    plt.title("Age-Specific Susceptible and Exposed")
    plt.xlabel("Time (days)")
    plt.ylabel("Population")
    plt.legend()
    plt.tight_layout()
    plt.show()


### Disease states(Im,Is,C,R,V)

In [10]:
def plot_disease_states(df):
    plt.figure(figsize=(10,6))
    for col in ["Im", "Is", "C", "R", "V"]:
        sns.lineplot(df, x="time", y=col, label=col)
        
    plt.title("Disease States (Age-independent)")
    plt.xlabel("Time (days)")
    plt.ylabel("Population")
    plt.legend()
    plt.tight_layout()
    plt.show()


### Mosquito compartments

In [11]:
def plot_mosquito(df):
    plt.figure(figsize=(10,6))
    for col in ["Sm", "Em", "Im_mosq"]:
        sns.lineplot(df, x="time", y=col, label=col)
        
    plt.title("Mosquito Compartments")
    plt.xlabel("Time (days)")
    plt.ylabel("Mosquito Population")
    plt.legend()
    plt.tight_layout()
    plt.show()


### Key Indicators

In [12]:
def plot_indicators(df):
    df["Total_Infectious"] = df["Im"] + df["Is"]
    df["Total_Susceptible"] = df["S1"] + df["S2"]
    df["Total_Exposed"] = df["E1"] + df["E2"]
    
    plt.figure(figsize=(10,6))
    for col in ["Total_Infectious", "C", "Total_Susceptible"]:
        sns.lineplot(df, x="time", y=col, label=col)
        
    plt.title("Key Epidemiological Indicators")
    plt.xlabel("Time (days)")
    plt.ylabel("Population")
    plt.legend()
    plt.tight_layout()
    plt.show()


In [14]:
def plot_chikungunya_results(df):
    plot_age_specific(df)
    plot_disease_states(df)
    plot_mosquito(df)
    plot_indicators(df)
